1. How to import PySpark and check the version?

In [5]:
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.master("local[1]").appName("SparkAssessment.com").getOrCreate()

print(spark.version)

24/09/19 15:11:18 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.command

3.5.2


In [47]:
from pyspark.sql.functions import monotonically_increasing_id,row_number,min,max
from pyspark.sql import Window,functions as F

2. How to convert the index of a PySpark DataFrame into a column?


In [19]:
df = spark.createDataFrame([
("Alice", 1),
("Bob", 2),
("Charlie", 3),
], ["Name", "Value"])

df.show()
df_with_index=df.withColumn("index",monotonically_increasing_id())
df_with_index.show()

#or
window_spec=Window.orderBy("value")
df_index=df.withColumn("index",row_number().over(window_spec)-1)
df_index.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+

+-------+-----+-----+
|   Name|Value|index|
+-------+-----+-----+
|  Alice|    1|    0|
|    Bob|    2|    1|
|Charlie|    3|    2|
+-------+-----+-----+

+-------+-----+-----+
|   Name|Value|index|
+-------+-----+-----+
|  Alice|    1|    0|
|    Bob|    2|    1|
|Charlie|    3|    2|
+-------+-----+-----+



24/09/19 15:24:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/19 15:24:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/19 15:24:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


3. How to combine many lists to form a PySpark DataFrame?

In [23]:
list1 = ["a", "b", "c", "d"]
list2 = [1, 2, 3, 4]
data=list(zip(list1,list2))
df=spark.createDataFrame(data,["c1","c2"])
df.show()

+---+---+
| c1| c2|
+---+---+
|  a|  1|
|  b|  2|
|  c|  3|
|  d|  4|
+---+---+



4. How to get the items of list A not present in list B?

In [25]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

rdd1=spark.sparkContext.parallelize(list_A)
rdd2=spark.sparkContext.parallelize(list_B)

res=rdd1.subtract(rdd2)
res.collect()

[2, 1, 3]

5.How to get the items not common to both list A and list B?

In [26]:
res1=rdd2.subtract(rdd1)
res_union=res.union(res1)
res_union.collect()

[2, 1, 3, 6, 8, 7]

6. How to get the minimum, 25th percentile, median, 75th, and max of a numeric column?

In [33]:
data = [("A", 10), ("B", 20), ("C", 30), ("D", 40), ("E", 50), ("F", 15), ("G", 28), ("H", 54), ("I", 41), ("J", 86)]
df = spark.createDataFrame(data, ["Name", "Age"])

df.show()
min_val=df.agg(min("age")).collect()[0][0]
max_val=df.agg(max("age")).collect()[0][0]

quantiles=df.approxQuantile("age",[0.25,0.50,0.75],0.01)

print("min value:",min_val)
print("25th quartile:",quantiles[0])
print("median:",quantiles[1])
print("75th quartile:",quantiles[2])
print("max val",max_val)

+----+---+
|Name|Age|
+----+---+
|   A| 10|
|   B| 20|
|   C| 30|
|   D| 40|
|   E| 50|
|   F| 15|
|   G| 28|
|   H| 54|
|   I| 41|
|   J| 86|
+----+---+

min value: 10
25th quartile: 20.0
median: 30.0
75th quartile: 50.0
max val 86


7. How to get frequency counts of unique items of a column?

In [51]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()
name_count=df.groupBy("job").count()
name_count.show()


+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+

+---------+-----+
|      job|count|
+---------+-----+
|Scientist|    2|
|   Doctor|    1|
| Engineer|    4|
+---------+-----+



8. How to keep only top 2 most frequent values as it is and replace everything else as ‘Other’?

In [52]:
top_2_jobs = name_count.orderBy(F.desc("count")).limit(2).select("job").rdd.flatMap(lambda x: x).collect()

# Replace other jobs with 'Other'
result_df = df.withColumn("job", F.when(df["job"].isin(top_2_jobs), df["job"]).otherwise("Other"))

# Show the resulting DataFrame
result_df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|    Other|
+----+---------+



9. How to Drop rows with NA values specific to a particular column?

In [53]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|NULL|
|   B| NULL| 123|
|   B|    3| 456|
|   D| NULL|NULL|
+----+-----+----+



In [55]:
df.dropna(subset="id").show()

+----+-----+---+
|Name|Value| id|
+----+-----+---+
|   B| NULL|123|
|   B|    3|456|
+----+-----+---+



10. How to rename columns of a PySpark DataFrame using two lists – one containing the old column names and the other containing the new column names?

In [58]:
# suppose you have the following DataFrame
df = spark.createDataFrame([(1, 2, 3), (4, 5, 6)], ["col1", "col2", "col3"])

# old column names
old_names = ["col1", "col2", "col3"]

# new column names
new_names = ["new_col1", "new_col2", "new_col3"]

df.show()

for old_name, new_name in zip(old_names, new_names):
    df = df.withColumnRenamed(old_name, new_name)
print("DataFrame with Renamed Columns:")
df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
+----+----+----+

DataFrame with Renamed Columns:
+--------+--------+--------+
|new_col1|new_col2|new_col3|
+--------+--------+--------+
|       1|       2|       3|
|       4|       5|       6|
+--------+--------+--------+

